In [72]:
import QuantLib as ql
from datetime import datetime as dt

https://www.implementingquantlib.com/2024/05/inflation-curves.html

## Assumptions

### Date

In [73]:
today = dt.today()
today = ql.Date(today.day, today.month, today.year)
ql.Settings.instance().evaluationDate = today

calendar = ql.Poland()
day_count = ql.ActualActual(ql.ActualActual.Bond)

### Bond

In [74]:
issue_date = ql.Date(25, 8, 2023)
maturity_date = ql.Date(25, 8, 2036)
settlement_days = 2
face_value = 1000.0
fixing_lag = ql.Period(3, ql.Months)
fixing_frequency = ql.Monthly
forecast_frequency = ql.Annual
coupon_rate = 0.02

## Inflation curve and index

In [75]:
# forecasts of the inflation
dates = [
    ql.Date(1, 1, 2024),
    ql.Date(1, 1, 2025),
    ql.Date(1, 1, 2026),
    ql.Date(1, 1, 2027),
    ql.Date(1, 1, 2028),
    ql.Date(1, 1, 2029),
    ql.Date(1, 1, 2030),
    ql.Date(1, 1, 2031),
    ql.Date(1, 1, 2032),
    ql.Date(1, 1, 2033),
    ql.Date(1, 1, 2034),
    ql.Date(1, 1, 2035),
    ql.Date(1, 1, 2036),
]

rates = [0.037, 0.043, 0.029, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025]

inflation_curve_handle = ql.ZeroInflationTermStructureHandle(
    ql.ZeroInflationCurve(today, calendar, day_count, fixing_lag, forecast_frequency, dates, rates)
)

# Define custom CPI index
cpi_index = ql.ZeroInflationIndex(
    "CPI", ql.CustomRegion('Poland', 'PL'), False, fixing_frequency, fixing_lag, ql.PLNCurrency(), inflation_curve_handle
)

In [76]:
historical_cpi_fixings = [
    (ql.Date(1, 7,  2024),  0.014),
    (ql.Date(1, 6,  2024),  0.001),
    (ql.Date(1, 5,  2024),  0.001),
    (ql.Date(1, 4,  2024),  0.011),
    (ql.Date(1, 3,  2024),  0.002),
    (ql.Date(1, 2,  2024),  0.003),
    (ql.Date(1, 1,  2024),  0.004),
    (ql.Date(1, 12, 2023),  0.001),
    (ql.Date(1, 11, 2023),  0.007),
    (ql.Date(1, 10, 2023),  0.003),
    (ql.Date(1, 9,  2023), -0.004),
    (ql.Date(1, 8,  2023),  0.000),
    (ql.Date(1, 7,  2023), -0.002),
    (ql.Date(1, 6,  2023),  0.000),
    (ql.Date(1, 5,  2023),  0.000),
    (ql.Date(1, 4,  2023),  0.007),
    (ql.Date(1, 3,  2023),  0.011),
    (ql.Date(1, 2,  2023),  0.012),
    (ql.Date(1, 1,  2023),  0.024),
]

for date, rate in historical_cpi_fixings:
    cpi_index.addFixing(date, rate)

## Bond

In [77]:
# Define the inflation-linked bond
schedule = ql.Schedule(
    issue_date,
    maturity_date,
    ql.Period(ql.Annual),
    calendar,
    ql.Unadjusted,
    ql.Unadjusted,
    ql.DateGeneration.Forward,
    False,
)
schedule.dates()

(Date(25,8,2023),
 Date(25,8,2024),
 Date(25,8,2025),
 Date(25,8,2026),
 Date(25,8,2027),
 Date(25,8,2028),
 Date(25,8,2029),
 Date(25,8,2030),
 Date(25,8,2031),
 Date(25,8,2032),
 Date(25,8,2033),
 Date(25,8,2034),
 Date(25,8,2035),
 Date(25,8,2036))

In [78]:
growth_only = False
interpolation = ql.CPI.Linear

inflation_bond = ql.CPIBond(
    settlement_days,
    face_value,
    growth_only,
    cpi_index.fixing(today),  # base CPI
    fixing_lag,
    cpi_index,  # CPI index
    interpolation,
    schedule,  # schedule
    [coupon_rate],  # fixed rates
    day_count,  # day count convention
    ql.Unadjusted,  # business day convention
    issue_date,
    calendar,
)

In [79]:
# Create the yield term structure based on fixed rate bonds
yield_curve = [
    ("ON", 0.05640, today + ql.Period(1, ql.Days)),
    ("PS1024", 0.03768, ql.Date(25, 10, 2024)),
    ("PS0425", 0.04049, ql.Date(25, 4,  2025)),
    ("DS0725", 0.04412, ql.Date(25, 7,  2025)),
    ("DS0726", 0.04691, ql.Date(25, 7,  2026)),
    ("PS1026", 0.04756, ql.Date(25, 10, 2026)),
    ("PS0527", 0.04778, ql.Date(25, 5,  2027)),
    ("DS0727", 0.04771, ql.Date(25, 7,  2027)),
    ("WS0428", 0.04780, ql.Date(25, 4,  2028)),
    ("PS0728", 0.04837, ql.Date(25, 7,  2028)),
    ("WS0429", 0.04871, ql.Date(25, 4,  2029)),
    ("PS0729", 0.04946, ql.Date(25, 7,  2029)),
    ("DS1029", 0.04854, ql.Date(25, 10, 2029)),
    ("DS1030", 0.04927, ql.Date(25, 10, 2030)),
    ("DS0432", 0.05001, ql.Date(25, 4,  2032)),
    ("DS1033", 0.05046, ql.Date(25, 10, 2033)),
    ("DS1034", 0.05105, ql.Date(25, 10, 2034)),
    ("WS0437", 0.05187, ql.Date(25, 4,  2037)),
    ("WS0447", 0.05225, ql.Date(25, 4,  2047)),
]

irs_curve = [
    ("ON", 0.05640, today + ql.Period(1, ql.Days)),
    ("1Y", 0.05515, today + ql.Period(1, ql.Years)),
    ("2Y", 0.04870, today + ql.Period(2, ql.Years)),
    ("3Y", 0.04497, today + ql.Period(3, ql.Years)),
    ("4Y", 0.04354, today + ql.Period(4, ql.Years)),
    ("5Y", 0.04300, today + ql.Period(5, ql.Years)),
    ("6Y", 0.04284, today + ql.Period(6, ql.Years)),
    ("7Y", 0.04294, today + ql.Period(7, ql.Years)),
    ("8Y", 0.04318, today + ql.Period(8, ql.Years)),
    ("9Y", 0.04343, today + ql.Period(9, ql.Years)),
    ("10Y", 0.04370, today + ql.Period(10, ql.Years)),
    ("15Y", 0.04529, today + ql.Period(15, ql.Years)),
    ("20Y", 0.04675, today + ql.Period(20, ql.Years)),
]

curve = yield_curve

interest_rate = ql.ForwardCurve(
    [point[2] for point in curve], [point[1] for point in curve], day_count, calendar
)

yield_curve = ql.YieldTermStructureHandle(interest_rate)

### Pricing

In [80]:
# Set up a pricing engine
bond_engine = ql.DiscountingBondEngine(yield_curve)
inflation_bond.setPricingEngine(bond_engine)

# Get the clean price
clean_price = inflation_bond.cleanPrice()
print(f"Clean Price: {clean_price:.2f}")

# Get the dirty price
dirty_price = inflation_bond.dirtyPrice()
print(f"Dirty Price: {dirty_price:.2f}")

# Get the yield
bond_yield = inflation_bond.bondYield(day_count, ql.Compounded, ql.Annual)
print(f"Yield: {bond_yield:.2%}")

Clean Price: 94.22
Dirty Price: 94.69
Yield: 5.00%


In [81]:
rate = ql.InterestRate(bond_yield, day_count, ql.Compounded, ql.Annual)
duration = ql.BondFunctions.duration(inflation_bond, rate, ql.Duration.Simple)
print(f'Duration: {duration:.2f}')

mduration = ql.BondFunctions.duration(inflation_bond, rate, ql.Duration.Modified)
print(f'Modified Duration: {mduration:.2f}')

Duration: 10.74
Modified Duration: 10.23


## Other